# Import Libraries

In [1]:
import importlib
import sys
sys.modules['imp'] = importlib

In [2]:
%load_ext autoreload
# %reload_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from IPython import display

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
params_cfg = {
    "action"   : "train_feat01",
    "feat_path": "/content/drive/MyDrive/house_predict/process/exps/data.npz",
    "seed"    : 42, # Set random seed
    "exp_dir" : os.path.abspath('/content/drive/MyDrive/house_predict/process/exps'),
    'exp_name': 'trainbase_27102025',
    "data_dir": os.path.abspath("/content/drive/MyDrive/house_predict/data"),
    "verbose" : True,
}

params_cfg.update(**{
    "save_dir": os.path.abspath(f'{params_cfg["exp_dir"]}/{params_cfg["exp_name"]}')
})

for v in params_cfg:
    print(f'+ {v}: {params_cfg[v]}')

globals().update(**params_cfg)

+ action: train_feat01
+ feat_path: /content/drive/MyDrive/house_predict/process/exps/data.npz
+ seed: 42
+ exp_dir: /content/drive/MyDrive/house_predict/process/exps
+ exp_name: trainbase_27102025
+ data_dir: /content/drive/MyDrive/house_predict/data
+ verbose: True
+ save_dir: /content/drive/MyDrive/house_predict/process/exps/trainbase_27102025


# Data Load

In [4]:
df_train = pd.read_csv(f'{data_dir}/train.csv')
df_test = pd.read_csv(f'{data_dir}/test.csv')

if params_cfg["verbose"]:
    print("-"*10, "information", "-"*10)
    print(f'train-col: {set(df_train.columns)}')
    print(f'test-col: {set(df_test.columns)}')
    print("Union:", set(df_train.columns).intersection(set(df_test.columns)))
    print("Difference:", set(df_train.columns).difference(set(df_test.columns)))

---------- information ----------
train-col: {'LotShape', 'FireplaceQu', '3SsnPorch', 'BsmtFinType1', 'HeatingQC', 'CentralAir', 'Alley', '1stFlrSF', 'ExterCond', 'GarageType', 'MoSold', 'Exterior1st', 'WoodDeckSF', 'BsmtFinSF1', 'MiscVal', 'YearRemodAdd', 'KitchenAbvGr', 'Neighborhood', 'LowQualFinSF', 'GarageQual', 'OverallQual', 'Utilities', 'YrSold', 'Street', 'SaleType', 'OpenPorchSF', 'Functional', 'PoolQC', 'BsmtFullBath', 'LotFrontage', 'GrLivArea', 'Fence', 'EnclosedPorch', 'TotRmsAbvGrd', 'GarageYrBlt', 'OverallCond', 'BldgType', 'PavedDrive', 'TotalBsmtSF', 'Fireplaces', 'GarageCond', 'ExterQual', 'LotArea', 'RoofMatl', 'LandSlope', 'FullBath', 'BsmtUnfSF', '2ndFlrSF', 'Condition1', 'GarageCars', 'Condition2', 'Electrical', 'HouseStyle', 'RoofStyle', 'KitchenQual', 'BsmtCond', 'MSSubClass', 'MasVnrType', 'Exterior2nd', 'BsmtHalfBath', 'SalePrice', 'BsmtFinType2', 'BsmtQual', 'LotConfig', 'BedroomAbvGr', 'BsmtFinSF2', 'HalfBath', 'MSZoning', 'ScreenPorch', 'GarageFinish', 'Po

# Processing

## Xử lí các dữ liệu thiếu trong việc phân tích EDA

In [5]:
df_output = pd.DataFrame()
df_output = df_train.copy()

### Xóa những cột trên 50% dữ liệu thiếu

In [6]:
missing_ratio = df_output.isnull().sum() / len(df_output)
cols_to_drop = missing_ratio[missing_ratio > 0.5].index
df_output = df_output.drop(cols_to_drop, axis=1)

In [7]:
num_cols = df_output.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df_output.select_dtypes(include=['object','category']).columns

### Xử lí missing value cho các cột num_cols và cat_cols

In [8]:
for col in num_cols:
    if df_output[col].isnull().sum() > 0:
        median_value = df_output[col].median()
        df_output[col] = df_output[col].fillna(median_value)


In [9]:
cols_fill_none = [
    'FireplaceQu', 'GarageFinish', 'GarageType', 'GarageQual', 'GarageCond',
    'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1', 'BsmtCond', 'BsmtQual'
]

for col in cols_fill_none:
    df_output[col] = df_output[col].fillna('None')

df_output['Electrical'] = df_output['Electrical'].fillna(df_output['Electrical'].mode()[0])


### Tạo các feature mới

In [10]:
# df_output['Qual_LivArea'] = df_output['OverallQual'] * df_output['TotalLiving']
df_output['GarageScore'] = df_output['GarageArea'] + df_output['GarageCars']
df_output['TotalAmenities'] = (
    df_output['Fireplaces'] + df_output['FullBath'] + df_output['KitchenAbvGr']
)
df_output['Loc_Style_Cond'] = (
    df_output['Neighborhood'] + '_' + df_output['HouseStyle'] + '_' + df_output['SaleCondition']
)
df_output['Qual_Exter_Overall'] = (
    df_output['ExterQual'] + '_' + df_output['OverallQual'].astype(str)
)


# Tổng diện tích sử dụng được
df_output['TotalArea'] = (
    df_output['GrLivArea'] + df_output['TotalBsmtSF'] +
    df_output['GarageArea'] + df_output['WoodDeckSF'] +
    df_output['OpenPorchSF']
)

# Diện tích trung bình mỗi phòng
df_output['AreaPerRoom'] = df_output['GrLivArea'] / (df_output['TotRmsAbvGrd'] + 1)

# Tuổi nhà và thời gian sửa
df_output['RemodAge'] = df_output['YrSold'] - df_output['YearRemodAdd']
df_output['AgeSinceBuilt'] = df_output['YrSold'] - df_output['YearBuilt']

# Tổng số phòng tắm (tính cả tầng hầm)
df_output['Baths_Total'] = (
    df_output['FullBath'] + 0.5 * df_output['HalfBath'] +
    df_output['BsmtFullBath'] + 0.5 * df_output['BsmtHalfBath']
)

# Đặc trưng tương tác giữa diện tích và chất lượng
df_output['Qual_LotArea'] = df_output['OverallQual'] * df_output['LotArea']
df_output['TotalArea_Qual'] = df_output['TotalArea'] * df_output['OverallQual']

# Thời gian từ lần sửa đến năm bán
df_output['SoldSinceRemod'] = df_output['YrSold'] - df_output['YearRemodAdd']


new_features = [
    'GarageScore', 'TotalAmenities',
    'Loc_Style_Cond', 'Qual_Exter_Overall', 'TotalArea', 'AreaPerRoom',
    'RemodAge', 'AgeSinceBuilt', 'Baths_Total', 'Qual_LotArea',
    'TotalArea_Qual', 'SoldSinceRemod'
]

print("✅ Tổng số feature mới được thêm vào df_output:", len(new_features))
print(new_features)


✅ Tổng số feature mới được thêm vào df_output: 12
['GarageScore', 'TotalAmenities', 'Loc_Style_Cond', 'Qual_Exter_Overall', 'TotalArea', 'AreaPerRoom', 'RemodAge', 'AgeSinceBuilt', 'Baths_Total', 'Qual_LotArea', 'TotalArea_Qual', 'SoldSinceRemod']


In [11]:
def preprocessing_feature_01(df_data, is_train=True, is_debug=True, **kwargs):
    """
    Hàm tiền xử lý dữ liệu mở rộng cho bài House Prices.
    - Điền missing value
    - Tạo feature mới
    - Encode dữ liệu dạng category bằng One-Hot Encoding
    """
    df_output = df_data.copy()

    missing_ratio = df_output.isnull().sum() / len(df_output)
    cols_to_drop = missing_ratio[missing_ratio > 0.5].index
    df_output = df_output.drop(cols_to_drop, axis=1)

    # ---- 1️⃣ Điền giá trị thiếu cho dạng số ----
    num_cols = df_output.select_dtypes(include=['int64', 'float64']).columns
    for col in num_cols:
        if df_output[col].isnull().sum() > 0:
            median_value = df_output[col].median()
            df_output[col] = df_output[col].fillna(median_value)

    # ---- 2️⃣ Điền giá trị thiếu cho dạng category ----
    cols_fill_none = [
        'FireplaceQu', 'GarageFinish', 'GarageType', 'GarageQual', 'GarageCond',
        'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1', 'BsmtCond', 'BsmtQual'
    ]
    for col in cols_fill_none:
        if col in df_output.columns:
            df_output[col] = df_output[col].fillna('None')

    if 'Electrical' in df_output.columns:
        df_output['Electrical'] = df_output['Electrical'].fillna(df_output['Electrical'].mode()[0])

    # ---- 3️⃣ Feature engineering ----
    if all(c in df_output.columns for c in ['GarageArea', 'GarageCars']):
        df_output['GarageScore'] = df_output['GarageArea'] + df_output['GarageCars']

    if all(c in df_output.columns for c in ['Fireplaces', 'FullBath', 'KitchenAbvGr']):
        df_output['TotalAmenities'] = (
            df_output['Fireplaces'] + df_output['FullBath'] + df_output['KitchenAbvGr']
        )

    if all(c in df_output.columns for c in ['Neighborhood', 'HouseStyle', 'SaleCondition']):
        df_output['Loc_Style_Cond'] = (
            df_output['Neighborhood'] + '_' +
            df_output['HouseStyle'] + '_' +
            df_output['SaleCondition']
        )

    if all(c in df_output.columns for c in ['ExterQual', 'OverallQual']):
        df_output['Qual_Exter_Overall'] = (
            df_output['ExterQual'] + '_' + df_output['OverallQual'].astype(str)
        )

    if all(c in df_output.columns for c in ['GrLivArea', 'TotalBsmtSF', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF']):
        df_output['TotalArea'] = (
            df_output['GrLivArea'] + df_output['TotalBsmtSF'] +
            df_output['GarageArea'] + df_output['WoodDeckSF'] + df_output['OpenPorchSF']
        )

    if all(c in df_output.columns for c in ['GrLivArea', 'TotRmsAbvGrd']):
        df_output['AreaPerRoom'] = df_output['GrLivArea'] / (df_output['TotRmsAbvGrd'] + 1)

    if all(c in df_output.columns for c in ['YrSold', 'YearRemodAdd']):
        df_output['RemodAge'] = df_output['YrSold'] - df_output['YearRemodAdd']

    if all(c in df_output.columns for c in ['YrSold', 'YearBuilt']):
        df_output['AgeSinceBuilt'] = df_output['YrSold'] - df_output['YearBuilt']

    bath_cols = ['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath']
    if all(c in df_output.columns for c in bath_cols):
        df_output['Baths_Total'] = (
            df_output['FullBath'] + 0.5 * df_output['HalfBath'] +
            df_output['BsmtFullBath'] + 0.5 * df_output['BsmtHalfBath']
        )

    if all(c in df_output.columns for c in ['OverallQual', 'LotArea']):
        df_output['Qual_LotArea'] = df_output['OverallQual'] * df_output['LotArea']

    if all(c in df_output.columns for c in ['TotalArea', 'OverallQual']):
        df_output['TotalArea_Qual'] = df_output['TotalArea'] * df_output['OverallQual']

    if all(c in df_output.columns for c in ['YrSold', 'YearRemodAdd']):
        df_output['SoldSinceRemod'] = df_output['YrSold'] - df_output['YearRemodAdd']

    cols_fill_mode = [
    'MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd',
    'KitchenQual', 'Functional', 'SaleType'
    ]

    for col in cols_fill_mode:
        if col in df_output.columns:
            df_output[col] = df_output[col].fillna(df_output[col].mode()[0])



    # ---- 5️⃣ Debug ----
    if is_debug:
        print("✅ Hoàn tất preprocessing_feature_01")
        print(f"🔹 Dữ liệu đầu ra: {df_output.shape[0]} hàng, {df_output.shape[1]} cột")
        new_cols = [col for col in df_output.columns if col not in df_data.columns]
        print(f"✨ Số lượng feature mới: {len(new_cols)}")
        print("🆕 Feature mới:", new_cols[:15], "..." if len(new_cols) > 15 else "")

    return df_output, None


In [12]:
def main_feat01(**kwargs):
  # load data
  df_train = pd.read_csv(f'{data_dir}/train.csv')
  df_test = pd.read_csv(f'{data_dir}/test.csv')
  # preprocessing
  df_output_train, _ = preprocessing_feature_01(df_train, is_train=True, is_debug=False)
  df_output_test, _ = preprocessing_feature_01(df_test, is_train=False, is_debug=False)

  # saving
  os.makedirs(save_dir, exist_ok=True)

  # Lưu trực tiếp DataFrame objects (cần allow_pickle=True khi load)
  np.savez(f'{save_dir}/data01.npz',
             train_data=df_output_train.values,
             test_data=df_output_test.values,
             train_columns=df_output_train.columns.values,  # Lưu tên cột train
             test_columns=df_output_test.columns.values,
             allow_pickle=True)

  print("Đã lưu DataFrame 01 với đầy đủ thông tin cột")

  kwargs.get('global_cfg', {}).update(**locals())

if params_cfg["action"] == "train_feat01":
    print("Runing ... [train_feat01]")
    main_feat01(global_cfg = globals())

Runing ... [train_feat01]
Đã lưu DataFrame 01 với đầy đủ thông tin cột
